## 개요

### 파이토치

- **GPU**에서 **텐서** 조작 및 **동적 신경망** 구축이 가능한 프레임워크

  - GPU : 빠른 병렬 연산을 가능하게 함

  - Tensor : 단일 데이터 형식으로 된 자료들의 다차원 행렬 / 변수 뒤에 .cuda() 를 사용하여 GPU 연산 수행
  
  - 동적 신경망 : 학습을 반복할 때마다 네트워크 수정이 가능한 신경망

### 텐서

- 텐서는 **1차원 배열** 형태여야만 메모리에 저장할 수 있음

- 변환된 1차원 배열을 스토리지(storage)라고 함

### 오프셋과 스트라이드

- 텐서 내 요소의 위치를 계산하기 위해 사용함

  - 오프셋 : 시작점으로부터의 거리
  
  - 스트라이드 : 인접한 요소 사이의 거리


In [200]:
import numpy as np

np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

- 요소 '6'까지의 오프셋 : 5

  - 첫 번째 인덱스부터 '6'까지의 총 5개의 요소가 있음을 의미함
<br><br>
- 스트라이드 : (3, 1)

  - 첫 번째 인덱스부터 '4'까지 도달하기 위해서는 3칸 건너뛰어야 함
  
  - 첫 번째 인덱스부터 '2'까지 도달하기 위해서는 1칸 건너뛰어야 함

---

## 기초 문법

In [201]:
import torch

### (1) 텐서 생성 및 변환

In [202]:
# 2차원 텐서 생성
torch.tensor([[1, 2], [3, 4]])

tensor([[1, 2],
        [3, 4]])

In [203]:
# GPU에 텐서 생성
torch.tensor([[1, 2], [3, 4]], device='cuda:0')

tensor([[1, 2],
        [3, 4]], device='cuda:0')

In [204]:
# dtype 설정한 텐서 생성
torch.tensor([[1, 2], [3, 4]], dtype=torch.float64)

tensor([[1., 2.],
        [3., 4.]], dtype=torch.float64)

In [205]:
# 텐서를 ndarray로 변환
tmp = torch.tensor([[1, 2], [3, 4]])

print(tmp)
print('\n')
print(tmp.numpy())

tensor([[1, 2],
        [3, 4]])


[[1 2]
 [3 4]]


In [206]:
# GPU 상의 텐서를 CPU 텐서로 변환한 후, ndarray로 변환
tmp = torch.tensor([[1, 2], [3, 4]], device='cuda:0')

print(tmp)
print('\n')
print(tmp.to('cpu').numpy())

tensor([[1, 2],
        [3, 4]], device='cuda:0')


[[1 2]
 [3 4]]


### (2) 텐서의 인덱스 조작

- 배열처럼 인덱스 지정 및 슬라이싱 사용 가능

In [208]:
tmp = torch.FloatTensor([1, 2, 3, 4, 5, 6, 7])

In [209]:
tmp[0], tmp[-1]

(tensor(1.), tensor(7.))

In [210]:
tmp[2:5], tmp[4:-1]

(tensor([3., 4., 5.]), tensor([5., 6.]))

### (3) 텐서의 차원 조작

- 가장 대표적인 방법 : **view**

  - numpy의 reshape과 유사함
<br><br>
- 텐서 결합 : stack, cat

- 차원 교환 : t, transpose

In [211]:
tmp = torch.tensor([[1, 2], [3, 4]])
tmp

tensor([[1, 2],
        [3, 4]])

In [ ]:
# 2x2
tmp.shape

torch.Size([2, 2])

In [213]:
# 4x1로 변형
tmp.view(4, 1)

tensor([[1],
        [2],
        [3],
        [4]])

In [214]:
# 1차원 벡터로 변형
tmp.view(-1)

tensor([1, 2, 3, 4])

In [215]:
# 1행으로 만들기 (열은 자동으로)
tmp.view(1, -1)

tensor([[1, 2, 3, 4]])

In [216]:
# 1열로 만들기 (행은 자동으로)
tmp.view(-1, 1)

tensor([[1],
        [2],
        [3],
        [4]])

- -1 : 알아서 유추

- (2x2) -> (1x?)

  - ? = 4

---

## 모델 정의

In [218]:
import torch.nn as nn

#### (1) 단순 신경망 정의

In [219]:
model = nn.Linear(in_features=1, out_features=1, bias=True)

#### (2) `nn.Module`을 상속하여 정의

  - `__init__()` : 모델에 사용될 모듈과 활성화 함수 등을 정의
  
  - `__forward__()` : 모델에서 실행되어야 하는 연산을 정의

In [220]:
class MLP(nn.Module):
    def __init__(self, inputs):
        super(MLP, self).__init__()
        self.layer = nn.Linear(inputs, 1) # 레이어
        self.activation = nn.Sigmoid() # 활성화 함수


    def forward(self, X):
        X = self.layer(X)
        X = self.activation(X)
        return X

#### (3) `nn.Sequential()` 신경망 정의

  - `__init()__` 에서 사용할 모델을 정의해 줄 뿐만 아니라,

  - `__forward()__` 에서 실행되어야 할 연산을 가독성 좋게 작성할 수 있음
  
  - 각 모듈을 순차적으로 실행함

In [221]:
class MLP(nn.Module):
    def __init__(self): # 모듈, 활성화 함수 정의
        super(MLP, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2))

        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=30, kernel_size=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2))

        self.layer3 = nn.Sequential(
            nn.Linear(in_features=30*5*5, out_features=10, bias=True),
            nn.ReLU(inplace=True))


    def forward(self, x): # 실행될 연산 정의
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(x.shape[0], -1)
        x = self.layer3(x)
        return x

In [222]:
# 인스턴스 생성
model = MLP()

In [223]:
# MLP 모델 하위에 있는 같은 레벨의 노드
list(model.children())

[Sequential(
   (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
   (1): ReLU(inplace=True)
   (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 ),
 Sequential(
   (0): Conv2d(64, 30, kernel_size=(5, 5), stride=(1, 1))
   (1): ReLU(inplace=True)
   (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 ),
 Sequential(
   (0): Linear(in_features=750, out_features=10, bias=True)
   (1): ReLU(inplace=True)
 )]

In [224]:
# 모델의 모든 노드
list(model.modules())

[MLP(
   (layer1): Sequential(
     (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
     (1): ReLU(inplace=True)
     (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
   )
   (layer2): Sequential(
     (0): Conv2d(64, 30, kernel_size=(5, 5), stride=(1, 1))
     (1): ReLU(inplace=True)
     (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
   )
   (layer3): Sequential(
     (0): Linear(in_features=750, out_features=10, bias=True)
     (1): ReLU(inplace=True)
   )
 ),
 Sequential(
   (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
   (1): ReLU(inplace=True)
   (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 ),
 Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1)),
 ReLU(inplace=True),
 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
 Sequential(
   (0): Conv2d(64, 30, kernel_size=(5, 5), stride=(1, 1))
   (1): ReLU(inplace=True)
   (2): MaxPool2d(kernel_size=2

---

## 파라미터 정의

- 모델을 학습시키기 전에 필요한 파라미터들을 정의함

#### (1) loss function : 출력(wx + b)과 정답(y) 사이의 오차를 측정함

  - 손실함수 값을 최소화하는 **weight**, **bias를** 찾는 것이 학습의 목표

    - BCELoss : 이진 분류

    - CrossEntropyLoss : 다중 분류
    
    - MSELoss : 회귀

#### (2) optimizer : 데이터와 손실함수를 바탕으로 모델의 업데이트 방법을 결정함

- `step()` : 전달받은 파라미터를 업데이트 함

- `zero_grad()` : 파라미터의 기울기를 0으로 만듦


#### (3) learning rate scheduler : 지정한 횟수의 epoch마다 learning rate를 감소시킴

- 학습 초기에는 큰 보폭으로 학습을 진행하다가, global minimum 근처에서 보폭을 줄여서 **최적 해**를 찾아갈 수 있도록 함

#### (4) metrics : 평가지표를 통해 학습과 테스트 과정을 모니터링함

In [226]:
# 모델 파라미터 정의
from torch.optim import optimizer

criterion = torch.nn.MSELoss() # 손실함수
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9) # 옵티마이저
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer=optimizer, lr_lambda=lambda epoch: 0.95 ** epoch) # 학습률 스케줄러

for epoch in range(1, 100+1):
    for x, y in dataloader:
        optimizer.zero_grad()

loss_fn(model(x), y).backward()
optimizer.step()
scheduler.step()

---

## 모델 학습

#### 학습이란 y = wx + b 에서 w와 b의 적절한 값을 찾아가는 과정

- (weight, bias)는 **임의의 값**으로 시작하고, 오차가 줄어들면서 global minimum에 도달할 때까지 계속 **업데이트** 함

#### `loss.backward()` : 기울기 값 계산 -> 새로운 기울기 값이 이전 기울기 값에 **누적**하여 계산됨

- RNN이 아닌 이상 누적 계산은 불필요함

- 따라서, `optimizer.zero_grad()` 사용하여 기울기를 초기화 시켜줌

#### 학습 절차

(1) 모델, 손실함수, 옵티마이저 정의

  - 기울기 초기화 : `optimizer.zero_grad()`

(2) forward 학습 (입력 -> 출력 계산)

  - `output = model(input)`

(3) 출력값과 정답값의 오차 계산

  - `loss = loss_fn(output, target)`

(4) backward 학습 (기울기 계산)

  - `loss.backward()`

(5) 기울기 업데이트

  - `optimizer.step()`

In [269]:
# 모델 학습
for epoch in range(num_epochs):
    yhat = model(x_train) # 모델의 출력값(예측값)
    loss = criterion(yhat, y_train) # 손실함수 계산(예측값과 정답값의 오차)
    optimizer.zero_grad() # 오차가 누적되지 않도록 기울기 초기화
    loss.backward() # 기울기 계산
    optimizer.step() # 기울기 업데이트

---

## 모델 평가

#### (1) 함수를 이용하여 모델 평가

In [ ]:
!pip install torchmetrics

In [271]:
import torchmetrics

In [272]:
preds = torch.randn(10, 5).softmax(dim=-1)
target = torch.randint(5, (10, ))

In [273]:
# 출력값(예측값)
preds

tensor([[0.0640, 0.0990, 0.2515, 0.1196, 0.4659],
        [0.0438, 0.1434, 0.0751, 0.1489, 0.5887],
        [0.6717, 0.0191, 0.1668, 0.0229, 0.1195],
        [0.1238, 0.2915, 0.1451, 0.3127, 0.1270],
        [0.2591, 0.0454, 0.0297, 0.5524, 0.1134],
        [0.6255, 0.0458, 0.1777, 0.0916, 0.0595],
        [0.0317, 0.2788, 0.1788, 0.0464, 0.4643],
        [0.0182, 0.4475, 0.1371, 0.3338, 0.0634],
        [0.4036, 0.0520, 0.1919, 0.2034, 0.1491],
        [0.0840, 0.6437, 0.0545, 0.1400, 0.0777]])

In [274]:
# 정답값
target

tensor([3, 4, 1, 3, 4, 4, 4, 2, 3, 1])

In [275]:
acc = torchmetrics.functional.accuracy(preds, target, 'MULTICLASS', num_classes=5)
acc

tensor(0.4000)

#### (2) 모듈을 이용하여 모델 평가

In [276]:
metric = torchmetrics.Accuracy('MULTICLASS', num_classes=5)

n_batches = 10
for i in range(n_batches):
    preds = torch.randn(10, 5).softmax(dim=-1)
    target = torch.randint(5, (10, ))

    acc = metric(preds, target)
    print(f"Accuracy on batch {i}: {acc}")

acc = metric.compute()
print(f"\nAccuracy on all data: {acc}")

Accuracy on batch 0: 0.30000001192092896
Accuracy on batch 1: 0.0
Accuracy on batch 2: 0.20000000298023224
Accuracy on batch 3: 0.20000000298023224
Accuracy on batch 4: 0.4000000059604645
Accuracy on batch 5: 0.10000000149011612
Accuracy on batch 6: 0.30000001192092896
Accuracy on batch 7: 0.10000000149011612
Accuracy on batch 8: 0.20000000298023224
Accuracy on batch 9: 0.0

Accuracy on all data: 0.18000000715255737
